In [1]:
%matplotlib inline

import sys
import os
import torch
import torch.nn as nn
import torch.optim as optim
import albumentations as A
import argparse
import matplotlib.pyplot as plt
import numpy as np
import cv2
from torch.utils.data import DataLoader
from albumentations.pytorch import ToTensorV2
from os.path import abspath, join, dirname
from tqdm import tqdm
import sklearn

sys.path.append(os.path.abspath(os.path.join("..", "unet")))
from dataset import COCODataset

WIDTH = 256
HEIGHT = 256
BATCH_SIZE = 16

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"DEVICE: {device}")

C:\Users\danie\.virtualenvs\BoneSegmentation-dpCXn3BL\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DEVICE: cuda:0


In [2]:
image_path = "D:\\Datasets\\Segmentation\\COCO\\images\\val2017"
mask_path = "D:\\Datasets\\Segmentation\\COCO\\masks\\val2017"

params = {
    "batch_size": BATCH_SIZE,
    "shuffle": True,
    "num_workers": 0
}

In [3]:
transform = A.Compose(
    [
#         A.Resize(height=HEIGHT, width=WIDTH),
        A.Resize(height=int(HEIGHT*1.13), width=int(WIDTH*1.13)),
        A.RandomCrop(height=HEIGHT, width=WIDTH),
        A.Rotate(limit=90, p=0.9),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        A.CLAHE(clip_limit=3.0, p=0.35),
        A.ColorJitter(p=0.3),
        A.GaussNoise(p=0.3),
        A.Normalize(
            mean=[0.4690, 0.4456, 0.4062],
            std=[0.2752, 0.2701, 0.2847],
            max_pixel_value=255.0
        ),
        ToTensorV2()
    ]
)

cocodata = COCODataset(image_path, mask_path, transform=transform)
cocoloader = DataLoader(cocodata, **params) 
# Mean: tensor([0.4690, 0.4456, 0.4062]), Std: tensor([0.2752, 0.2701, 0.2847])
# mean, std = cocodata.get_dataset_mean_and_std(cocoloader)
# print(f"Mean: {mean}, Std: {std}")

# class_weights = cocodata.get_class_weights(cocoloader)
# print(class_weights)

12it [00:12,  1.03s/it]


ValueError: classes should include all valid labels that can be in y

In [ ]:
# data (n_batch, 3, height, width), target (n_batch, height, width)
data, targets = next(iter(cocoloader))

print("Targets Shape: ", targets.shape)
print("Data Shape: ", data.shape)


ax_size = BATCH_SIZE**0.5
assert ax_size.is_integer() # make sure batch size perfect square
ax_size = int(ax_size)

f, axarr = plt.subplots(ax_size, ax_size, figsize=(25,25))
count = 0
for i in range(ax_size):
    for j in range(ax_size):
        image = data[count]
        print(image.shape)
        fig_img = torch.moveaxis(image, 0, -1)
        axarr[i,j].imshow(fig_img)
        plt.axis("off")
        print(
            f"""\nimage shape: {image.dtype},
            \n\tmean: {torch.mean(image, dim=[1,2])},
            \n\tstd: {torch.std(image, dim=[1,2])}"""
        )
        count += 1
        
